In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13862012857425490955, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 380070527285233892
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [14]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "Xception_InceptionV3_descriptors"

In [10]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [11]:
# from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

In [12]:
input_tensor = Input(shape = input_shape)  

base_model1=InceptionV3(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model2=Xception(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)

x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

merge = concatenate([x1, x2])
predictions = Dense(num_classes, activation='softmax')(merge)

model = Model(inputs=input_tensor,outputs=predictions)

In [13]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 block1_conv1
56 batch_normalization_1

In [15]:
c1 = model.layers[11].output 
c1 = GlobalAveragePooling2D()(c1)       

c2 = model.layers[21].output
c2 = GlobalAveragePooling2D()(c2)       

c3 = model.layers[28].output
c3 = GlobalAveragePooling2D()(c3)       

c4 = model.layers[51].output
c4 = GlobalAveragePooling2D()(c4) 

c5 = model.layers[84].output
c5 = GlobalAveragePooling2D()(c5) 

c6 = model.layers[103].output
c6 = GlobalAveragePooling2D()(c6) 

c7 = model.layers[117].output
c7 = GlobalAveragePooling2D()(c7) 

c8 = model.layers[129].output
c8 = GlobalAveragePooling2D()(c8) 

c9 = model.layers[143].output
c9 = GlobalAveragePooling2D()(c9) 

c10 = model.layers[162].output
c10 = GlobalAveragePooling2D()(c10) 

c11 = model.layers[210].output
c11 = GlobalAveragePooling2D()(c11) 

c12 = model.layers[258].output
c12 = GlobalAveragePooling2D()(c12) 

c13 = model.layers[306].output
c13 = GlobalAveragePooling2D()(c13) 

c14 = model.layers[356].output
c14 = GlobalAveragePooling2D()(c14) 

c15 = model.layers[377].output
c15 = GlobalAveragePooling2D()(c15) 

c16 = model.layers[415].output
c16 = GlobalAveragePooling2D()(c16) 

c17 = model.layers[421].output
c17 = GlobalAveragePooling2D()(c17) 

con = concatenate([c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15, c16])

bottleneck_final_model = Model(inputs=model.input, outputs=con)

In [16]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [17]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [18]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [19]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [20]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [21]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [22]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [23]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [24]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.9, 'lr': 0.001, 'beta_2': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.2122 - acc: 0.4701 - val_loss: 1.1089 - val_acc: 0.5400
Epoch 2/15
 - 4s - loss: 1.3907 - acc: 0.3820 - val_loss: 1.1767 - val_acc: 0.5909
Epoch 3/15
 - 4s - loss: 1.4969 - acc: 0.3289 - val_loss: 1.4108 - val_acc: 0.4291
Epoch 4/15
 - 4s - loss: 1.5798 - acc: 0.2705 - val_loss: 1.3724 - val_acc: 0.4639
Epoch 5/15
 - 4s - loss: 1.6398 - acc: 0.2373 - val_loss: 1.6566 - val_acc: 0.2312
Epoch 00005: early stopping
14958/14958 [==============================] - 1s 37us/step
Validation Accuracy: 23.1181%
Validation Loss: 1.6566366956169902
Test Accuracy: 0.23037266165856532
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.4, 'lr': 0.001, 'beta_2': 0.99}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 

Epoch 3/15
 - 4s - loss: 2.5643 - acc: 0.1863 - val_loss: 1.7138 - val_acc: 0.3208
Epoch 4/15
 - 4s - loss: 2.4098 - acc: 0.2035 - val_loss: 1.6665 - val_acc: 0.3327
Epoch 5/15
 - 4s - loss: 2.2863 - acc: 0.2175 - val_loss: 1.6400 - val_acc: 0.3144
Epoch 6/15
 - 4s - loss: 2.1646 - acc: 0.2375 - val_loss: 1.6261 - val_acc: 0.3034
Epoch 7/15
 - 4s - loss: 2.0768 - acc: 0.2568 - val_loss: 1.6191 - val_acc: 0.3051
Epoch 00007: early stopping
14958/14958 [==============================] - 1s 38us/step
Validation Accuracy: 30.5054%
Validation Loss: 1.6190969470924736
Test Accuracy: 0.42996022978347326
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'lr': 1e-07, 'beta_2': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 2.6206 - acc: 0.1933 - val_loss: 1.8110 - val_acc: 0.2089
Epoch 2/15
 - 4s - loss: 2.4251 - acc: 0.2101 -

Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 11.9117 - acc: 0.2537 - val_loss: 12.4695 - val_acc: 0.2264
Epoch 2/15
 - 4s - loss: 10.7818 - acc: 0.3171 - val_loss: 11.4114 - val_acc: 0.2906
Epoch 3/15
 - 4s - loss: 9.2727 - acc: 0.3535 - val_loss: 10.8135 - val_acc: 0.3202
Epoch 4/15
 - 4s - loss: 3.6274 - acc: 0.3326 - val_loss: 1.8599 - val_acc: 0.3422
Epoch 5/15
 - 4s - loss: 2.4178 - acc: 0.3201 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 6/15
 - 4s - loss: 5.0328 - acc: 0.2847 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 7/15
 - 4s - loss: 4.8469 - acc: 0.2978 - val_loss: 12.6236 - val_acc: 0.2168
Epoch 00007: early stopping
14958/14958 [==============================] - 1s 40us/step
Validation Accuracy: 21.6807%
Validation Loss: 12.623578551938024
Test Accuracy: 0.2154956547356017
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {

Epoch 15/15
 - 4s - loss: 0.7550 - acc: 0.7414 - val_loss: 0.9819 - val_acc: 0.5783
14958/14958 [==============================] - 1s 40us/step
Validation Accuracy: 57.8286%
Validation Loss: 0.9818873497585129
Test Accuracy: 0.7167476800707026
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.6, 'lr': 1e-05, 'beta_2': 0.5}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.1835 - acc: 0.5832 - val_loss: 1.0415 - val_acc: 0.5551
Epoch 2/15
 - 4s - loss: 0.7261 - acc: 0.7500 - val_loss: 0.9025 - val_acc: 0.6241
Epoch 3/15
 - 4s - loss: 0.6166 - acc: 0.7865 - val_loss: 0.8046 - val_acc: 0.6672
Epoch 4/15
 - 4s - loss: 0.5577 - acc: 0.8065 - val_loss: 0.7997 - val_acc: 0.6710
Epoch 5/15
 - 4s - loss: 0.5167 - acc: 0.8226 - val_loss: 0.7294 - val_acc: 0.7088
Epoch 6/15
 - 4s - loss: 0.4841 - acc: 0.8319 - val_loss: 0.7394 - val_acc:

Epoch 3/15
 - 4s - loss: 1.1424 - acc: 0.5863 - val_loss: 1.2620 - val_acc: 0.4529
Epoch 4/15
 - 4s - loss: 1.0195 - acc: 0.6445 - val_loss: 1.1923 - val_acc: 0.4880
Epoch 5/15
 - 4s - loss: 0.9410 - acc: 0.6737 - val_loss: 1.1347 - val_acc: 0.5158
Epoch 6/15
 - 4s - loss: 0.8836 - acc: 0.6971 - val_loss: 1.0917 - val_acc: 0.5321
Epoch 7/15
 - 4s - loss: 0.8329 - acc: 0.7143 - val_loss: 1.0509 - val_acc: 0.5552
Epoch 8/15
 - 4s - loss: 0.8009 - acc: 0.7264 - val_loss: 1.0208 - val_acc: 0.5700
Epoch 9/15
 - 4s - loss: 0.7647 - acc: 0.7399 - val_loss: 0.9966 - val_acc: 0.5821
Epoch 10/15
 - 4s - loss: 0.7347 - acc: 0.7488 - val_loss: 0.9671 - val_acc: 0.5989
Epoch 11/15
 - 4s - loss: 0.7156 - acc: 0.7532 - val_loss: 0.9485 - val_acc: 0.6061
Epoch 12/15
 - 4s - loss: 0.6942 - acc: 0.7629 - val_loss: 0.9341 - val_acc: 0.6114
Epoch 13/15
 - 4s - loss: 0.6795 - acc: 0.7646 - val_loss: 0.9133 - val_acc: 0.6233
Epoch 14/15
 - 4s - loss: 0.6639 - acc: 0.7729 - val_loss: 0.9039 - val_acc: 0.6268

...

Epoch 5/15
 - 9s - loss: 0.3706 - acc: 0.8686 - val_loss: 0.6879 - val_acc: 0.7456
Epoch 6/15
 - 9s - loss: 0.3556 - acc: 0.8740 - val_loss: 0.6757 - val_acc: 0.7391
Epoch 00006: early stopping
14958/14958 [==============================] - 2s 122us/step
Validation Accuracy: 73.9069%
Validation Loss: 0.6756649340853704
Test Accuracy: 0.8538076299896892
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.0001, 'beta_2': 0.999}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 20s - loss: 0.7144 - acc: 0.7493 - val_loss: 0.7497 - val_acc: 0.6985
Epoch 2/15
 - 8s - loss: 0.4771 - acc: 0.8322 - val_loss: 0.6849 - val_acc: 0.7329
Epoch 3/15
 - 9s - loss: 0.4200 - acc: 0.8524 - val_loss: 0.7098 - val_acc: 0.7140
Epoch 4/15
 - 9s - loss: 0.3979 - acc: 0.8585 - val_loss: 0.6559 - val_acc: 0.7518
Epoch 5/15
 - 9s - loss: 0.3737 - acc: 0.86

Parameters testing:  {'beta_1': 0.5, 'lr': 0.0001, 'beta_2': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 19s - loss: 0.7101 - acc: 0.7475 - val_loss: 0.7819 - val_acc: 0.6954
Epoch 2/15
 - 9s - loss: 0.4715 - acc: 0.8301 - val_loss: 0.7426 - val_acc: 0.7138
Epoch 3/15
 - 9s - loss: 0.4209 - acc: 0.8513 - val_loss: 0.7107 - val_acc: 0.7413
Epoch 4/15
 - 9s - loss: 0.3961 - acc: 0.8596 - val_loss: 0.7557 - val_acc: 0.7276
Epoch 5/15
 - 9s - loss: 0.3808 - acc: 0.8691 - val_loss: 0.7374 - val_acc: 0.7390
Epoch 6/15
 - 8s - loss: 0.3703 - acc: 0.8731 - val_loss: 0.7236 - val_acc: 0.7546
Epoch 7/15
 - 9s - loss: 0.3594 - acc: 0.8765 - val_loss: 0.7242 - val_acc: 0.7590
Epoch 8/15
 - 9s - loss: 0.3544 - acc: 0.8794 - val_loss: 0.7426 - val_acc: 0.7552
Epoch 9/15
 - 9s - loss: 0.3510 - acc: 0.8828 - val_loss: 0.8513 - val_acc: 0.7276
Epoch 10/15
 - 9s - loss: 0.3475 - acc: 0.8867 - val_loss: 0.8474 - val_acc: 0.7482
Epoch 00010: early stopping
14958/14958 [===========

Epoch 5/15
 - 9s - loss: 0.3681 - acc: 0.8693 - val_loss: 0.7136 - val_acc: 0.7276
Epoch 6/15
 - 9s - loss: 0.3531 - acc: 0.8745 - val_loss: 0.6507 - val_acc: 0.7504
Epoch 7/15
 - 9s - loss: 0.3421 - acc: 0.8762 - val_loss: 0.6795 - val_acc: 0.7311
Epoch 8/15
 - 9s - loss: 0.3335 - acc: 0.8834 - val_loss: 0.6505 - val_acc: 0.7504
Epoch 9/15
 - 9s - loss: 0.3265 - acc: 0.8840 - val_loss: 0.6875 - val_acc: 0.7469
Epoch 00009: early stopping
14958/14958 [==============================] - 2s 143us/step
Validation Accuracy: 74.6891%
Validation Loss: 0.6875068840585318
Test Accuracy: 0.8581528943879806
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.2, 'lr': 0.0001, 'beta_2': 0.4}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 0.7042 - acc: 0.7495 - val_loss: 0.7480 - val_acc: 0.7024
Epoch 2/15
 - 9s - loss: 0.4775 - acc: 0.8326

Parameters testing:  {'beta_1': 0.6, 'lr': 0.0001, 'beta_2': 0.8}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 0.7023 - acc: 0.7516 - val_loss: 0.7966 - val_acc: 0.6845
Epoch 2/15
 - 9s - loss: 0.4728 - acc: 0.8337 - val_loss: 0.7541 - val_acc: 0.6967
Epoch 3/15
 - 9s - loss: 0.4153 - acc: 0.8535 - val_loss: 0.7313 - val_acc: 0.7200
Epoch 4/15
 - 10s - loss: 0.3933 - acc: 0.8611 - val_loss: 0.6397 - val_acc: 0.7603
Epoch 5/15
 - 10s - loss: 0.3727 - acc: 0.8687 - val_loss: 0.7370 - val_acc: 0.7288
Epoch 6/15
 - 9s - loss: 0.3611 - acc: 0.8732 - val_loss: 0.6598 - val_acc: 0.7649
Epoch 7/15
 - 11s - loss: 0.3518 - acc: 0.8785 - val_loss: 0.6700 - val_acc: 0.7469
Epoch 8/15
 - 9s - loss: 0.3409 - acc: 0.8818 - val_loss: 0.7511 - val_acc: 0.7376
Epoch 9/15
 - 9s - loss: 0.3371 - acc: 0.8839 - val_loss: 0.7116 - val_acc: 0.7492
Epoch 00009: early stopping
14958/14958 [==============================] - 2s 128us/step
Validation Accuracy: 74.9231%
Validation Los

Epoch 13/15
 - 10s - loss: 0.3913 - acc: 0.8666 - val_loss: 0.6746 - val_acc: 0.7383
Epoch 14/15
 - 9s - loss: 0.3876 - acc: 0.8687 - val_loss: 0.6696 - val_acc: 0.7386
Epoch 15/15
 - 10s - loss: 0.3777 - acc: 0.8706 - val_loss: 0.6559 - val_acc: 0.7464
14958/14958 [==============================] - 2s 148us/step
Validation Accuracy: 74.6356%
Validation Loss: 0.6558518740552116
Test Accuracy: 0.8508616880247459
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.9, 'lr': 0.0001, 'beta_2': 0.3}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 1.5468 - acc: 0.6706 - val_loss: 0.7614 - val_acc: 0.7002
Epoch 2/15
 - 10s - loss: 0.6207 - acc: 0.7713 - val_loss: 0.7710 - val_acc: 0.6941
Epoch 3/15
 - 10s - loss: 0.6231 - acc: 0.7657 - val_loss: 0.7235 - val_acc: 0.6976
Epoch 4/15
 - 10s - loss: 0.6053 - acc: 0.7657 - val_loss: 0.7415 